In [ ]:
import numpy as np
import pandas as pd
import random
import json

import sklearn.metrics as met
import matplotlib.pyplot as plt

In [ ]:
rev = []
with open('Software_5.json') as f:
  for line in f:
    rev.append(json.loads(line))

#main dataframe
test = pd.DataFrame(columns=['Summary and Review', 'Rating'])

#adding data into dataframe 
for i in range(len(rev)):
    if('overall' in rev[i] and 'reviewText' in rev[i] and 'summary' in rev[i]):
        test.loc[i] = [rev[i]['summary'] + ' ' + rev[i]['reviewText'], rev[i]['overall']]
#function to turn ratings into two class
def rate(x):
    if(x>3):
        return 1
    elif(x<3):
        return 0
    else:
        return random.randint(0,1)
test['Rating'] = test['Rating'].apply(rate)
# import re
# def removeNum(x):
#     x = re.sub(r'[0-9]+|_', '', x)
#     x = " . ".join(x.split('.')).split()
#     from nltk.sentiment.util import mark_negation
#     x = mark_negation(x)
#     x = " ".join(x)
#     return x
# test['Summary and Review'] = test['Summary and Review'].apply(removeNum)

In [ ]:
"""
Performance Validation Methods
"""

def plt_confusionMatrix(classifier, X_test, Y_test):
    """
    Given a trained classifer and test values will plot a confusion matrix.
    """
    matrix = met.plot_confusion_matrix(classifier, X_test, Y_test, display_labels=['negative', 'positive'], cmap=plt.cm.Blues)
    plt.show(matrix)

def plt_precisionRecall(classifier, X_test, Y_test):
    """
    Given a trained classifers and test values will plot Precision-Recall curve with average precision(AP) score.
    """
    disp = met.plot_precision_recall_curve(classifier, X_test, Y_test)
    plt.show(disp)

In [ ]:
rev[0]
test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token= RegexpTokenizer(r'[a-zA-Z0-9]+')

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', min_df=50, tokenizer=token.tokenize)
text_t2 = tfidf.fit_transform(test['Summary and Review'] )

cvector = CountVectorizer(stop_words='english', ngram_range=(1,1))
text_t = cvector.fit_transform(test['Summary and Review'])

In [ ]:
len(cvector.get_feature_names())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_t, test['Rating'], test_size=0.25, random_state=5)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)

#from sklearn.naive_bayes import BernoulliNB
#BNB = BernoulliNB()
#BNB.fit(X_train, Y_train)


#from sklearn.naive_bayes import GaussianNB
#GNB = GaussianNB()
#GNB.fit(X_train.todense(), Y_train)


#from sklearn.naive_bayes import ComplementNB
#CNB = ComplementNB()
#CNB.fit(X_train, Y_train)

from sklearn import metrics
predicted = MNB.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, Y_test)

print(str('{:04.2f}'.format(accuracy_score*100))+'%')

In [ ]:
plt_confusionMatrix(MNB, X_test, Y_test)

In [ ]:
plt_precisionRecall(MNB, X_test, Y_test)